# Public Key Cryptography

Learn asymmetric encryption using RSA public/private key pairs.

In [1]:
from pathlib import Path

from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA

# Search for key files starting from current directory and moving up
def find_project_root():
    current = Path.cwd()
    # Try current directory first
    if (current / "my_keypair").exists():
        return current
    # Try parent directory
    if (current.parent / "my_keypair").exists():
        return current.parent
    # Try two levels up (for nested structures)
    if (current.parent.parent / "my_keypair").exists():
        return current.parent.parent
    return current  # Default to current if not found

PROJECT_FOLDER = find_project_root()
print(f"Looking for keypairs in folder: {PROJECT_FOLDER}")

PRIVATE_KEY_FILE = PROJECT_FOLDER / "my_keypair"
PUBLIC_KEY_FILE = PROJECT_FOLDER / "my_keypair.pub"

assert PRIVATE_KEY_FILE.exists(), f"Private key not found at {PRIVATE_KEY_FILE}. CWD: {Path.cwd()}"
assert PUBLIC_KEY_FILE.exists(), f"Public key not found at {PUBLIC_KEY_FILE}"

print(f"✓ Found private key: {PRIVATE_KEY_FILE}")
print(f"✓ Found public key: {PUBLIC_KEY_FILE}")


Looking for keypairs in folder: /workspaces/ECBS-5147-Data-Engineering-Cloud-Computing-Managed-AI-Services
✓ Found private key: /workspaces/ECBS-5147-Data-Engineering-Cloud-Computing-Managed-AI-Services/my_keypair
✓ Found public key: /workspaces/ECBS-5147-Data-Engineering-Cloud-Computing-Managed-AI-Services/my_keypair.pub


Load the private key from file
The private key must be kept secret and secure
Extract the public key from our private key
The public key can be freely shared with anyone


In [2]:
with open(PRIVATE_KEY_FILE, "r", encoding="utf8") as key_file:
    private_key = RSA.import_key(key_file.read())

public_key = private_key.publickey()
print(f"Public key:\n{public_key.export_key().decode('utf-8')}")


Public key:
-----BEGIN PUBLIC KEY-----
MIIBojANBgkqhkiG9w0BAQEFAAOCAY8AMIIBigKCAYEA5bcQjBRgiWnILU0RYcCi
Mq5bvatGEIBEEG6hZydErG4/qL4T5+TVrdWrKicyt5Zbie7jJuexGZcfx6TT4Iqt
zLH1xrKpUU25A8bh07DXQgVxVhf9/BDpQyZwRlortIUiIOYhOhYmHy60pHo9YjLd
z30aYRukEBuHRgR/4bbxKWcXQHn0s+YLl8fvVY00Ath29L8xHZxscA9U8Z4x+Yx7
D/ZEK7HfwTdwEt12qCQzw93qexPEuhYwDXcKZ6SzufR/UI+vE/m/fflBpDMVG/Ox
HqrlSSeFznOJWs+qaIHfhazvkc/dkvGxxQru8BWEBoKxJWOGiPa5PGvVREOHIlsX
TfWwnhosmyQr1594ISMFE6f93j6vPQIudpmEKN7fD6YRsQDqfB6vVs87f2zeQVPt
0Pq5b+4ibcLjoKc1GdMe3WWK+OA9I40dYrmHjdxCgclPHCwqLCaHZeMV5Nq7uLZK
By3qxOWG63LxX2J7va0Q9//mf4+EQQsYp6CKGuOBxxdnAgMBAAE=
-----END PUBLIC KEY-----


Message to be encrypted - must be converted to bytes
Create a cipher object using the public key for encryption
Encrypt our message - only someone with the private key can decrypt it
Save the encrypted message to a file


In [3]:
short_secret_message = "My Secret Message".encode("utf-8")

public_key_cipher = PKCS1_OAEP.new(public_key)

encrypted_message = public_key_cipher.encrypt(short_secret_message)
print("Encrypted message:")
print(encrypted_message.hex())

ENCRYPTED_MESSAGE_FILE = PROJECT_FOLDER / "encrypted_message.bin"
with open(ENCRYPTED_MESSAGE_FILE, "wb") as f:
    f.write(encrypted_message)


Encrypted message:
106f4af6c82c029af2ba37c08f0772b66f7030d77a1b43e4419e92353ba5ca9662270128936dcf2dd1e39ea79bc6f4dbe94f0df898635b452b7e287fa86014cad2e0f4f5d8a7d97e5464d7368bca410510a336e2eacd2cb102221a784ed5474ef723b4f6d08304a3edfd2c0aa0cce5a114da3e5169f9ea4fcef9ae0f2ccf6b0274d73ee5e54e5e7a38b8f4e381a2d64926397fe229e08e7dea71302ea96665989e627c4cd49ffbc3df379317d3465702a42e870a40aa511e6f2b8ab91efd990cf7d6b7dd13198a437b32d497b9dfda3af7c646a54565b97bee8042647c6fe31ea684725a4f26ef3bde96b98729c94c3cfe8bf52046365a22c9e177f61a7a4582007e7b30c201d9d6b7707eada750ce0fe9f4410da3d167e4e1ad7fa475910274c0224012f10cb1f6e5f44214c80cfae1481d51eda8d44051dc534a90738bbea206fdb1cba3ea0889b990a7644d8cea36f837efb1849c383d2d471cde75d0950a04dbb3e1c0d362fc6204f0be848471906faed99db53910eff1074cdcd40130a3


Read the encrypted message from file
Create a cipher object using the private key for decryption
Decrypt the message using the private key
This demonstrates asymmetric encryption where:
1. Messages are encrypted with a public key and can only be decrypted with the matching private key
2. The public key can be freely shared
3. The private key must be kept secret
4. This is more secure than symmetric encryption as the decryption key (private key) is never shared


In [4]:
with open(ENCRYPTED_MESSAGE_FILE, "rb") as f:
    encrypted_message_from_file = f.read()

private_key_cipher = PKCS1_OAEP.new(private_key)

decrypted_message = private_key_cipher.decrypt(encrypted_message_from_file)
print(f"Decrypted message: {decrypted_message.decode('utf-8')}")



Decrypted message: My Secret Message
